In [1]:
# Korean Sentence Splitter 설치
!pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# KPF-BERT-NER의 ner class 모음
!wget https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py

--2024-07-26 00:15:24--  https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11245 (11K) [text/plain]
Saving to: ‘label.py’

label.py            100%[===================>]  10.98K  --.-KB/s    in 0s      

2024-07-26 00:15:24 (75.3 MB/s) - ‘label.py’ saved [11245/11245]



### (코랩 작업시) 기사 데이터(csv), valid_ner.py 업로드

In [6]:
import pandas as pd

df = pd.read_csv("2024_norm_split.csv")
print(len(df))
df.head()

73668


,id,title,date,content
0,122846231_0,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반..."
1,122846231_1,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,장비 발전(조달) 부문에 집중됐던 것으로 전해졌다. 커우젠원(구건문) 대만정치대 ...
2,122846362_0,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,"13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭..."
3,122846362_1,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,역 방향으로 향하는 통일로 5차로에서 추돌사고가 발생해 도로가 한때 부분 통제됐다....
4,122846400_0,"'자산 301조원' 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...


In [7]:
from transformers import AutoTokenizer, BertForTokenClassification, logging
logging.set_verbosity_error()
import sys, os, torch
import numpy as np
import label # label.py를 업로드 한 후
import kss

print(torch.cuda.is_available()) # 리소스를 GPU 계열로 선택한 이후

True


In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("KPF/KPF-bert-ner")
model = AutoModelForTokenClassification.from_pretrained("KPF/KPF-bert-ner")

In [9]:
def ner_predict(text):
    text = text.replace('\n','')      # 개행된 부분을 합치고
    model.to("cuda")                  # KPFBert NER 모델을 GPU에 올리고

    sents = kss.split_sentences(text) # kss로 문장 분리
    decoding_ner_sentence = ""        # NER 태깅이 된 문장을 저장할 문자열
    word_list = list()                # 추출된 단어, 라벨을 저장할 리스트
    pred_str = list()                 # 예측된 라벨을 저장할 리스트

    for idx, sent in enumerate(sents):

      ## ---------------------------------- 문장을 토큰화하고 텐서로 변환하여 모델 입력으로 넣어 예측을 진행 ---------------------------------- ##
        sent = sent.replace(" ", "-") # 공백을 하이픈(단어 경계)으로 교체
        test_tokenized = tokenizer(sent, return_tensors="pt") # 토크나이징, 텐서로 변경

        # input_ids, attention_mask, type_ids를 GPU에 올리고
        test_input_ids = test_tokenized["input_ids"].to("cuda")
        test_attention_mask = test_tokenized["attention_mask"].to("cuda")
        test_token_type_ids = test_tokenized["token_type_ids"].to("cuda")

        # 모델 입력으로 사용할 input을 딕셔너리 형태로 선언
        inputs = {
            "input_ids" : test_input_ids,
            "attention_mask" : test_attention_mask,
            "token_type_ids" : test_token_type_ids
        }
        # 입력 길이가 512 토큰을 넘으면 512를 기준으로 잘라서 다시 GPU에 올려서 모델로 predict 진행
        if inputs['input_ids'].size()[1] > 512: 
            cnt = int(inputs['input_ids'].size()[1]) # 입력 길이를 cnt로 저장하고

            # GPU에 있던 input들을 CPU의 numpy로 변환
            inp_np = inputs['input_ids'].cpu().numpy()
            att_np = inputs['attention_mask'].cpu().numpy()
            tok_np = inputs['token_type_ids'].cpu().numpy()

            # 입력 길이(cnt > 512)만큼
            for i in range(cnt):
                # 512길이를 단위로 하여 블록으로 자르고
                slice_inp = inp_np[0][(i*512):((i+1)*512)]
                slice_att = att_np[0][(i * 512):((i + 1) * 512)]
                slice_tok = tok_np[0][(i * 512):((i + 1) * 512)]
                # 2차원으로 변형
                slice_inp = slice_inp.reshape(1, len(slice_inp))
                slice_att = slice_att.reshape(1, len(slice_att))
                slice_tok = slice_tok.reshape(1, len(slice_tok))
                # 텐서로 변환하고
                slice_inp = torch.tensor(slice_inp)
                slice_att = torch.tensor(slice_att)
                slice_tok = torch.tensor(slice_tok)
                # 쪼갠 블록을 GPU에 올려서
                slice_inp = torch.tensor(slice_inp).to("cuda")
                slice_att = torch.tensor(slice_att).to("cuda")
                slice_tok = torch.tensor(slice_tok).to("cuda")
                # 모델 입력에 넣을 것을 해당 블록으로 정하고
                slice_inputs = {
                    "input_ids": slice_inp,
                    "attention_mask": slice_att,
                    "token_type_ids": slice_tok
                }
                # 모델의 예측을 진행
                outputs = model(**slice_inputs)
                token_predictions = outputs[0].argmax(dim=2) # 각 토큰의 예측된 라벨(index값)을 얻고
                token_prediction_list = token_predictions.squeeze(0).tolist() # 예측된 라벨(index값)을 리스트로 변환
                pred = [label.id2label[l] for l in token_prediction_list] # label.py의 id2label을 사용해서 예측된 라벨의 인덱스를 실제 라벨로 변환
                pred_str = np.concatenate((pred_str, pred)) # 예측된 라벨을 pred_str에 누적
        else:
            # 512 토큰을 넘지 않는 경우 그대로 모델의 예측을 진행
            outputs = model(**inputs)
            token_predictions = outputs[0].argmax(dim=2)
            token_prediction_list = token_predictions.squeeze(0).tolist()
            pred_str = [label.id2label[l] for l in token_prediction_list]
        
        ## ---------------------------------- 문장을 토큰화하고 토큰 문자열 리스트로 변환하여 모델 예측 결과를 해석 ---------------------------------- ##
        tt_tokenized = tokenizer(sent).encodings[0].tokens # 토크나이저로 해당 문장을 다시 토크나이징, 토큰 문자열 리스트로 변환
 
        is_prev_entity = False        # 이전 토큰이 엔티티였는가 여부
        prev_entity_tag = ""          # 이전 엔티티 토큰을 저장
        is_there_B_before_I = False   # I- 태그 앞에 B- 태그가 있는지 여부
        _word = ""                    # 현재 단어를 저장

        for i, (token, pred) in enumerate(zip(tt_tokenized, pred_str)): # 토큰 문자열과 예측 결과를 하나씩 묶어서
            if i == 0 or i == len(pred_str) - 1: # 토큰이 맨 처음(CLS)이거나 맨 끝(SEP)이면 패스
                continue
            # 서브워드를 나타내기 위해 표현된 #을 없애고, 단어 경계를 나타내기 위해 표현된 -을 다시 공백으로 변환
            token = token.replace('#', '').replace("-", " ")  
            if token == "": # 토큰이 비었으면 패스
                continue
            
            # 토큰의 예측 라벨이 B-로 시작하면(엔티티 시작)
            if 'B-' in pred: 
                # 이전 토큰들에 대한 처리 관련
                # 이전 엔티티가 있는 경우(이전 엔티티는 완성시키고 새로운 태그의 시작으로 처리해야함)
                if is_prev_entity is True: 
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' # 이전 엔티티 태그를 닫고(지금 B로 새로 시작되니깐) 
                    word_list.append({"word" : _word, "label" : prev_entity_tag}) # 기존 단어와 완성된 라벨로 word_list에 추가
                    _word = "" # 새로 단어를 받을 수 있게 초기화

                # 현재 토큰에 대한 처리 관련
                # 토큰의 맨 앞 캐릭터가 공백이면(단어 경계) --> 이 부분 처리가 약간 이해가 안됨
                if token[0] == ' ': 
                    token = list(token)     # 토큰을 리스트화 하고
                    token[0] = ' <'         # 첫 번째 부분을 엔티티 태그를 여는 부분(<)으로 시작
                    token = ''.join(token)  
                    decoding_ner_sentence += token
                    _word += token
                else:
                    decoding_ner_sentence += '<' + token  # 문장에는 <와 token을 붙이고
                    _word += token                        # 단어에는 < 없이 토큰만 추가

                is_prev_entity = True           # 이전 태그가 있음을 표시
                prev_entity_tag = pred[2:]      # 'B-', 'I-'와 같은 부분을 제거하고 그 뒤에 있는 태그로 지정
                is_there_B_before_I = True      # B가 이전에 있었음을 표시
            
            # 토큰의 예측 라벨이 I-로 시작하면(엔티티 중간)
            elif 'I-' in pred:
                decoding_ner_sentence += token  # 현재 토큰을 문장에 그대로 붙이고
                _word += token                  # 기존 단어에도 현재 토큰을 붙이고

                if is_there_B_before_I is True: # 이전에 B가 있었으면
                    is_prev_entity = True       # 이전 엔티티가 있다고 지정
            
            # 토큰의 예측 라벨이 O인 경우(엔티티가 아님)
            else:
                if is_prev_entity is True: # 이전 엔티티가 있는 경우
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token # 이전 엔티티 태그를 닫고(지금 O가 와서 연결이 안되니) 
                    is_prev_entity = False # 이전 엔티티 초기화
                    is_there_B_before_I = False # B-가 있지도 않은 상태
                    word_list.append({"word" : _word, "label" : prev_entity_tag}) # 완성된 이전 엔티티를 리스트에 추가
                    _word = "" # 현재 처리중인 단어 초기화
                else: # 이전 엔티티가 없는 경우
                    decoding_ner_sentence += token # 그냥 지금 토큰을 그대로 추가
    return word_list # 결과로는 추출된 NER들의 리스트만 반환

In [10]:
from valid_ner import valid_ner_code
import re
from tqdm import tqdm

# ner_origin = [] # NER 원본
ner_custom = [] # 전처리된 NER
contents = df['content'].tolist()
for content in tqdm(contents):
    valid_ner = []
    try:
      ner_results = ner_predict(content)
      # ner_origin.append(ner_results)

      for ner_result in ner_results:
          if ner_result['label'] in list(valid_ner_code.keys()): # valid한 NER 클래스이면
              if (ner_result['label'] == 'PS_NAME') and (len(ner_result['word']) == 1):
                  continue
              if ner_result['word'] == "대표":
                  continue
              if '월' in ner_result['word'] or '일' in ner_result['word']: # '12월 31일중국' 이런 식의 오류가 있다
                  pattern1 = r'\d{1,2}월'
                  pattern2 = r'\d{1,2}일'

                  # 정규 표현식을 사용하여 날짜 부분 제거
                  ner_result['word'] = re.sub(pattern1, '', ner_result['word'])
                  ner_result['word'] = re.sub(pattern2, '', ner_result['word']).replace("  ", " ").strip()
              # 유효한 NER만 뽑아서 리스트에 넣음
              valid_ner.append(ner_result['word'])
    except Exception as e:
      print(content[:30])
      print(e)
      print()
      valid_ner.append('-')
    if valid_ner == []: # valid한 NER 클래스가 하나도 없어서 빈 리스트일 수도 있음
      valid_ner.append('-')
    ner_custom.append(valid_ner)

  0%|          | 0/73668 [00:00<?, ?it/s]WARNING:root:Oh! You have mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 73668/73668 [2:08:07<00:00,  9.58it/s]


In [11]:
len(ner_custom)

73668

In [12]:
for x in ner_custom[:10]:
  print(x)

['리상푸', '전국방부장', '리부장', '해군', '둥쥔', '중국', '군', '중국', '국방부장', '대만', '필리핀', '남중국해', '중국', '신화통신', '중국', '국회', '로켓군', '부사령원', '중앙군', '연합참모부', '부참모장인', '장전중', '장진중', '미사일', '로켓군']
['커우젠원', '구건문', '대만정치대', '주임', '전국인대', '중국', '중앙군사위', '중국', '리상푸', '李상복', '국방부장', '장관', '둥쥔', '동군', '인민해방군', '해군', '사령원', '해군참모총장', '국방부장', '산둥성', '옌타이', '해군', '부장', '인민해방군', '전략', '미사일', '로켓군', '주임', '부장', '인민해방군']
['서울', '기상청', '서울', '한반도', '강원', '서울', '광진구', '마을버스', '종로구', '서대문구']
['통일로', '5차로', '천호대교', '북단', '강원', '춘천시', '트럭', '강원', '삼척시', '아래', '구급대원', '평창군', '소방대원', '기상청', '북쪽', '남쪽', '기상청']
['아르노', 'LVMH', '회장', '베이조스', '게이츠', '머어이재용', '회장', '일론', '프랑스', '루이뷔통모에에네시', 'LVMH', '베르나르', '아르노', '회장', '블룸버그', '일론', '머스크', '테슬라', '최고', 'CEO', '머스크', 'LVMH', '아르노', '회장', 'LVMH', '제프']
['아마존', '게이츠', '마이크로소프트', 'MS', '스티브 발머', 'MS', 'CEO', '프랑스', '로레알', '프랑수아즈', '베탕쿠르', '메예르스', '블룸버그', '로레알', '12위메예르스', '한국인', '이재용', '삼성전자', '회장', '엔비디아', '젠슨 황', 'CEO', '손정의', '일본', '소프트뱅크', '회장']
['나이지리아', '대우건설', '한국인', '대우건설', '

In [15]:
cnt = 0
for x in ner_custom:
  if (x == []) or (x[0] == '-'):
      cnt += 1
print(cnt)

400


- 생각보다 NER 추출이 되지 않은게 400개나 되어 직접 값을 확인해보기로 함

In [ ]:
for idx in range(len(ner_custom)):
  if (ner_custom[idx] == []) or (ner_custom[idx][0] == '-'):
      print(contents[idx])

- 임의의 데이터 한 줄 뽑아서 NER 실행 및 결과 확인
- NER 함수 내부에 문제가 있는 것은 아니고 valid_ner에서 배제한 DT(날짜), QT(수량) 등의 불필요한 NER 클래스여서 배제한 것으로 파악

In [17]:
text = "금리 급등 시기에 늘린 단기 채권 올해 만기 몰려… 1년새 10조 증가 기업-등급별 본격 옥석 가리기 전망 올해 만기가 돌아오는 회사채 규모가 70조 원에 육박하며 역대 최대인 것으로 나타났다. 부동산 프로젝트파이낸싱(PF) 부실 위기가 커지면서 회사채 시장에서 업종과 등급에 따라 '옥석 가리기'가 본격화할 것으로 전망된다. 지난해 12월 31일 금융투자업계에 따르면 올해 만기가 돌아오는 일반 회사채 규모는 69조8592억 원으로 집계됐다. 이는 역대 최대 규모로 지난해 만기 물량(58조6028억 원)보다 10조 원 이상 많은 수준이다. 카드나 캐피털 회사들이 운영 자금을 확보하기 위해 발행한 여신전문금융채(여전채) 만기 물량도 82조9534억 원으로 역대 최대다. 올해 회사채 및 여전채 만기 물량이 크게 늘어난 데는 2022년 이후 급등한 시중금리가 큰 영향을 미쳤다. 장기 채권 금리가 높아지"
ner_results = ner_predict(text)
for x in ner_results:
  print(x)

{'word': '올해', 'label': 'DT_YEAR'}
{'word': '1년새', 'label': 'DT_DURATION'}
{'word': '10조', 'label': 'QT_PRICE'}
{'word': '올해', 'label': 'DT_YEAR'}
{'word': '70조', 'label': 'QT_PRICE'}
{'word': '지난해', 'label': 'DT_OTHERS'}
{'word': '12월31일', 'label': 'DT_DAY'}
{'word': '올해', 'label': 'DT_YEAR'}
{'word': '69조8592억', 'label': 'QT_PRICE'}
{'word': '지난해', 'label': 'DT_YEAR'}
{'word': '58조6028억', 'label': 'QT_PRICE'}
{'word': '원10조 원', 'label': 'QT_PRICE'}
{'word': '82조9534억', 'label': 'QT_PRICE'}
{'word': '올해', 'label': 'DT_YEAR'}
{'word': '2022년', 'label': 'DT_YEAR'}


- 데이터 프레임에 저장하기 위해 리스트를 공백으로 구분된 문자열로 변환

In [21]:
save_form = [' '.join(x) for x in ner_custom]
for x in save_form[:10]:
  print(x)

리상푸 전국방부장 리부장 해군 둥쥔 중국 군 중국 국방부장 대만 필리핀 남중국해 중국 신화통신 중국 국회 로켓군 부사령원 중앙군 연합참모부 부참모장인 장전중 장진중 미사일 로켓군
커우젠원 구건문 대만정치대 주임 전국인대 중국 중앙군사위 중국 리상푸 李상복 국방부장 장관 둥쥔 동군 인민해방군 해군 사령원 해군참모총장 국방부장 산둥성 옌타이 해군 부장 인민해방군 전략 미사일 로켓군 주임 부장 인민해방군
서울 기상청 서울 한반도 강원 서울 광진구 마을버스 종로구 서대문구
통일로 5차로 천호대교 북단 강원 춘천시 트럭 강원 삼척시 아래 구급대원 평창군 소방대원 기상청 북쪽 남쪽 기상청
아르노 LVMH 회장 베이조스 게이츠 머어이재용 회장 일론 프랑스 루이뷔통모에에네시 LVMH 베르나르 아르노 회장 블룸버그 일론 머스크 테슬라 최고 CEO 머스크 LVMH 아르노 회장 LVMH 제프
아마존 게이츠 마이크로소프트 MS 스티브 발머 MS CEO 프랑스 로레알 프랑수아즈 베탕쿠르 메예르스 블룸버그 로레알 12위메예르스 한국인 이재용 삼성전자 회장 엔비디아 젠슨 황 CEO 손정의 일본 소프트뱅크 회장
나이지리아 대우건설 한국인 대우건설 나이지리아 바이엘사주 한국인 운전사 총 나이지리아 한국인 대우건설 지사 포트하커트시 정부 대우건설 조승일
친부모 6주양쪽 다리 토니 허젤 영국 영국 BBC 찰스 3세 국왕 남동부 켄트 허젤 허젤 경친부모 귀 양 다리 무릎 아래 부모 징역 허젤 부모 톰 무어 경이 신종 코로나바이러스 감염증 코로나19
평택 서울 경찰 일본 변호사 경찰 서울경찰청 34분경경기 평택시 일본 변호사 하라다 가쿠우에 폭탄 카제드론 기자 일본어 한국어 서울 영등포구 국세청 검찰청 경찰청 국방부 경찰 일본
가수 싸이 유튜브 K팝 유튜브 싸이 피네이션 미국 핫100 유튜브 유튜브


In [22]:
df['NER'] = save_form
df.columns

Index(['id', 'title', 'date', 'content', 'NER'], dtype='object')

In [23]:
df.head()

,id,title,date,content,NER
0,122846231_0,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반...",리상푸 전국방부장 리부장 해군 둥쥔 중국 군 중국 국방부장 대만 필리핀 남중국해 중...
1,122846231_1,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,장비 발전(조달) 부문에 집중됐던 것으로 전해졌다. 커우젠원(구건문) 대만정치대 ...,커우젠원 구건문 대만정치대 주임 전국인대 중국 중앙군사위 중국 리상푸 李상복 국방부...
2,122846362_0,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,"13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭...",서울 기상청 서울 한반도 강원 서울 광진구 마을버스 종로구 서대문구
3,122846362_1,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,역 방향으로 향하는 통일로 5차로에서 추돌사고가 발생해 도로가 한때 부분 통제됐다....,통일로 5차로 천호대교 북단 강원 춘천시 트럭 강원 삼척시 아래 구급대원 평창군 소...
4,122846400_0,"'자산 301조원' 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,아르노 LVMH 회장 베이조스 게이츠 머어이재용 회장 일론 프랑스 루이뷔통모에에네시...


In [25]:
df.to_csv("2024_norm_split_ner.csv", index=False)

In [26]:
new = pd.read_csv("2024_norm_split_ner.csv")
new.head()

,id,title,date,content,NER
0,122846231_0,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반...",리상푸 전국방부장 리부장 해군 둥쥔 중국 군 중국 국방부장 대만 필리핀 남중국해 중...
1,122846231_1,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,20240101,장비 발전(조달) 부문에 집중됐던 것으로 전해졌다. 커우젠원(구건문) 대만정치대 ...,커우젠원 구건문 대만정치대 주임 전국인대 중국 중앙군사위 중국 리상푸 李상복 국방부...
2,122846362_0,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,"13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭...",서울 기상청 서울 한반도 강원 서울 광진구 마을버스 종로구 서대문구
3,122846362_1,"서울 12cm 기습 폭설… ""이런 눈 또 온다""",20240101,역 방향으로 향하는 통일로 5차로에서 추돌사고가 발생해 도로가 한때 부분 통제됐다....,통일로 5차로 천호대교 북단 강원 춘천시 트럭 강원 삼척시 아래 구급대원 평창군 소...
4,122846400_0,"'자산 301조원' 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,아르노 LVMH 회장 베이조스 게이츠 머어이재용 회장 일론 프랑스 루이뷔통모에에네시...
